In [4]:
!pip install -q tensorflow==2.12.0 keras==2.12.0 opencv-python-headless
from google.colab import drive
drive.mount('/content/drive')

# Project paths
DRIVE_BASE = '/content/drive/MyDrive/vitamin_vision'
DATA_DIR   = f'{DRIVE_BASE}/vitamins_detection'
TRAIN_DIR  = f'{DATA_DIR}/vitamins_detection'
TEST_DIR   = f'{DATA_DIR}/vitamins_detection'
MODEL_DIR  = f'{DRIVE_BASE}/models'

import os
os.makedirs(MODEL_DIR, exist_ok=True)
print("✅ Paths set:")
print("Train:", TRAIN_DIR)
print("Test :", TEST_DIR)
print("Model:", MODEL_DIR)

ERROR: Could not find a version that satisfies the requirement tensorflow==2.12.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.12.0
Mounted at /content/drive
✅ Paths set:
Train: /content/drive/MyDrive/vitamin_vision/vitamins_detection/vitamins_detection
Test : /content/drive/MyDrive/vitamin_vision/vitamins_detection/vitamins_detection
Model: /content/drive/MyDrive/vitamin_vision/models


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=(0.8,1.2),
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_ds = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=(128,128), batch_size=16, class_mode='categorical'
)
val_ds = test_datagen.flow_from_directory(
    TEST_DIR, target_size=(128,128), batch_size=16, class_mode='categorical'
)
print("Classes:", train_ds.class_indices)


Found 8968 images belonging to 5 classes.
Found 8968 images belonging to 5 classes.
Classes: {'vitaminA': 0, 'vitaminB': 1, 'vitaminC': 2, 'vitaminD': 3, 'vitaminE': 4}


In [6]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models, optimizers

base = VGG19(weights='imagenet', include_top=False, input_shape=(128,128,3))
for layer in base.layers: layer.trainable = False

x = layers.Flatten()(base.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
out = layers.Dense(5, activation='softmax')(x)

model = models.Model(inputs=base.input, outputs=out)
model.compile(optimizer=optimizers.Adam(1e-4),
              loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv4 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     4,194,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             

 Total params: 24,351,813 (92.89 MB)

 Trainable params: 4,327,429 (16.51 MB)

 Non-trainable params: 20,024,384 (76.39 MB)

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

best_model_path  = f'{MODEL_DIR}/vitamin_model_best.h5'
final_model_path = f'{MODEL_DIR}/vitamin_model_final.h5'

cb_ckpt = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, verbose=1)
cb_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1)
cb_lr   = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
callbacks = [cb_ckpt, cb_stop, cb_lr]


In [8]:
EPOCHS = 5
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
561/561 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.3550 - loss: 1.5367
Epoch 1: val_loss improved from inf to 1.18825, saving model to /content/drive/MyDrive/vitamin_vision/models/vitamin_model_best.h5


561/561 ━━━━━━━━━━━━━━━━━━━━ 3557s 6s/step - accuracy: 0.3550 - loss: 1.5365 - val_accuracy: 0.5256 - val_loss: 1.1883 - learning_rate: 1.0000e-04
Epoch 2/5
561/561 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 0.4753 - loss: 1.2859
Epoch 2: val_loss improved from 1.18825 to 1.10215, saving model to /content/drive/MyDrive/vitamin_vision/models/vitamin_model_best.h5


561/561 ━━━━━━━━━━━━━━━━━━━━ 116s 206ms/step - accuracy: 0.4753 - loss: 1.2859 - val_accuracy: 0.5666 - val_loss: 1.1021 - learning_rate: 1.0000e-04
Epoch 3/5
561/561 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.5056 - loss: 1.2076
Epoch 3: val_loss improved from 1.10215 to 1.09307, saving model to /content/drive/MyDrive/vitamin_vision/models/vitamin_model_best.h5


561/561 ━━━━━━━━━━━━━━━━━━━━ 116s 206ms/step - accuracy: 0.5056 - loss: 1.2076 - val_accuracy: 0.5727 - val_loss: 1.0931 - learning_rate: 1.0000e-04
Epoch 4/5
561/561 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.5201 - loss: 1.1698
Epoch 4: val_loss improved from 1.09307 to 1.02332, saving model to /content/drive/MyDrive/vitamin_vision/models/vitamin_model_best.h5


561/561 ━━━━━━━━━━━━━━━━━━━━ 116s 208ms/step - accuracy: 0.5201 - loss: 1.1698 - val_accuracy: 0.5982 - val_loss: 1.0233 - learning_rate: 1.0000e-04
Epoch 5/5
561/561 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.5416 - loss: 1.1424
Epoch 5: val_loss improved from 1.02332 to 0.98814, saving model to /content/drive/MyDrive/vitamin_vision/models/vitamin_model_best.h5


561/561 ━━━━━━━━━━━━━━━━━━━━ 119s 211ms/step - accuracy: 0.5416 - loss: 1.1424 - val_accuracy: 0.6117 - val_loss: 0.9881 - learning_rate: 1.0000e-04
Restoring model weights from the end of the best epoch: 5.


In [9]:
model.save(final_model_path)
print("✅ Final model saved at:", final_model_path)

from google.colab import files
files.download(best_model_path)   # download the best model to your PC


✅ Final model saved at: /content/drive/MyDrive/vitamin_vision/models/vitamin_model_final.h5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
from tensorflow.keras.preprocessing import image
import numpy as np

idx_to_class = {v:k for k,v in train_ds.class_indices.items()}

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(128,128))
    x = image.img_to_array(img)/255.
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)[0]
    return idx_to_class[np.argmax(pred)], float(np.max(pred))

# Example usage:
# label, prob = predict_image('/content/sample.jpg')
# print(label, prob)
